In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
RANDOM_VEC_SIZE = 50

In [3]:
sess = tf.InteractiveSession()

In [4]:
def generator(z, is_train=True, out_channel_dim=1):
    alpha = 0.2
    fc_size = 128
    
    with tf.variable_scope('generator', reuse=False if is_train==True else True):
        fc_1 = tf.layers.dense(z, 1024)
        batch_norm_1 = tf.layers.batch_normalization(fc_1, training=is_train)
        lrelu_1 = tf.nn.leaky_relu(batch_norm_1, alpha)
                
        fc_2 = tf.layers.dense(fc_1, 7 * 7 * fc_size)    
        
        deconv_3 = tf.reshape(fc_2, [-1, 7, 7, fc_size])
        batch_norm_3 = tf.layers.batch_normalization(deconv_3, training=is_train)
        lrelu_3 = tf.nn.leaky_relu(batch_norm_3, alpha)
        
        deconv_4 = tf.layers.conv2d_transpose(lrelu_3, fc_size // 2, 5, 2, padding="SAME")
        batch_norm_4 = tf.layers.batch_normalization(deconv_4, training=is_train)
        lrelu_4 = tf.nn.leaky_relu(batch_norm_4, alpha)
        
        deconv_5 = tf.layers.conv2d_transpose(lrelu_4, fc_size // 4, 5, 2, padding="SAME")
        batch_norm_5 = tf.layers.batch_normalization(deconv_5, training=is_train)
        lrelu_5 = tf.nn.leaky_relu(batch_norm_5, alpha)
        
        print("Shapes:")
        print(lrelu_3.shape)
        print(lrelu_4.shape)
        print(lrelu_5.shape)
        
        logits = tf.layers.conv2d(lrelu_5, out_channel_dim, 5, padding="SAME")
        
        return tf.tanh(logits)

In [5]:
def discrim(images, reuse=False, is_train=True, num_channels=1):
    alpha = 0.2
    
    
    #I have no idea why the variable naming convention suddenly changed here
    with tf.variable_scope("discrim", reuse=reuse):
        conv1 = tf.layers.conv2d(images, 256, 5, 1, "VALID")
        lrelu1 = tf.nn.leaky_relu(conv1, alpha)
        
        conv2 = tf.layers.conv2d(lrelu1, 128, 5, 2, "SAME")
        batch_norm2 = tf.layers.batch_normalization(conv2, training=is_train)
        lrelu2 = tf.nn.leaky_relu(batch_norm2, alpha)
        
        conv3 = tf.layers.conv2d(lrelu2, 64, 5, 1, "SAME")
        batch_norm3 = tf.layers.batch_normalization(conv3, training=is_train)
        lrelu3 = tf.nn.leaky_relu(batch_norm3, alpha)
        
        print("DISCRIM")
        print(lrelu1.shape)
        print(lrelu2.shape)
        print(lrelu3.shape)
        flat_lrelu3 = tf.reshape(lrelu3, [-1, 6**2 *64])
        
        fc1 = tf.layers.dense(flat_lrelu3, 512)
        logits = tf.layers.dense(fc1, 1) 
        
        return logits, tf.sigmoid(logits)  
        

In [6]:
def model_loss(input_real, input_z, num_channels=1):
    label_smoothing = 0.9
    
    fake_out = generator(input_z)
    G_sample_image = tf.summary.image("Deep_GAN", fake_out)
    
    real_logits, real_probs = discrim(tf.reshape(input_real, [-1, 28, 28, num_channels]))
    fake_logits, fake_probs = discrim(fake_out, True)
    
    D_loss_real = tf.reduce_mean(
        tf.nn.sigmoid_cross_entropy_with_logits(logits=real_logits, labels=label_smoothing * tf.ones_like(real_logits)))
    D_loss_fake = tf.reduce_mean(
        tf.nn.sigmoid_cross_entropy_with_logits(logits=fake_logits, labels=tf.zeros_like(fake_logits)))
    
    D_accur_real = tf.reduce_mean(tf.round(real_probs))
    D_accur_fake = tf.reduce_mean(tf.round(1 - fake_probs))
    
    D_accur = ((D_accur_real + D_accur_fake) / 2, D_accur_real, D_accur_fake)
    
    D_loss = D_loss_real + D_loss_fake
    
    G_loss = tf.reduce_mean(
        tf.nn.sigmoid_cross_entropy_with_logits(logits=fake_logits, labels=label_smoothing * tf.ones_like(fake_logits)))
    
    return D_loss, D_accur, G_loss, G_sample_image

In [7]:
def opt(D_loss, G_loss, learning_rate, beta1, weight_decay=0.01):
    t_vars = tf.trainable_variables()
    
    D_loss += tf.add_n([tf.nn.l2_loss(var) for var in t_vars if var.name.startswith("discrim")]) * weight_decay
    d_train = tf.train.AdamOptimizer(learning_rate, beta1=beta1).minimize(
        D_loss, var_list=[var for var in t_vars if var.name.startswith("discrim")])
    
    
    G_loss += tf.add_n([tf.nn.l2_loss(var) for var in t_vars if var.name.startswith("generator")]) * weight_decay
    g_train = tf.train.AdamOptimizer(learning_rate, beta1=beta1).minimize(
        G_loss, var_list=[var for var in t_vars if var.name.startswith("generator")])
    
    return d_train, g_train

In [8]:
Z = tf.placeholder(tf.float32, [None, RANDOM_VEC_SIZE])
X = tf.placeholder(tf.float32, [None, 784])

D_loss, D_accur, G_loss, G_sample_image = model_loss(X, Z)
D_trainer, G_trainer = opt(D_loss, G_loss, 2e-5, 0.5)

Shapes:
(?, 7, 7, 128)
(?, 14, 14, 64)
(?, 28, 28, 32)
DISCRIM
(?, 24, 24, 256)
(?, 12, 12, 128)
(?, 12, 12, 64)
DISCRIM
(?, 24, 24, 256)
(?, 12, 12, 128)
(?, 12, 12, 64)


In [9]:
def sample_Z(batch_size):
    return np.random.uniform(-1, 1, [batch_size, RANDOM_VEC_SIZE])

np.random.seed(0xDEADBEEF)

In [10]:
tf.global_variables_initializer().run()

summary_writer = tf.summary.FileWriter("tmp/logs", sess.graph)

In [ ]:
BATCH_SIZE = 25
FAKE_BATCH_SIZE = 25

DISCR_ITERS = 1
GEN_ITERS = 100

for ITER in range(100000):
    batch, _ = mnist.train.next_batch(BATCH_SIZE)
    fake_batch = sample_Z(FAKE_BATCH_SIZE)
    
    for _ in range(DISCR_ITERS):
        D_trainer.run(feed_dict={Z: fake_batch, X: batch})
    for _ in range(GEN_ITERS):
        G_trainer.run(feed_dict={Z: fake_batch, X: batch})
    
    if ITER % 10 == 0:
        if ITER % 100 == 0:
            
            batch, _ = mnist.train.next_batch(BATCH_SIZE)
            fake_batch = sample_Z(FAKE_BATCH_SIZE)
            
            D_train_loss = D_loss.eval(feed_dict={Z: fake_batch, X: batch})
            G_train_loss = G_loss.eval(feed_dict={Z: fake_batch, X: batch})
            
            D_train_accur = [part.eval(feed_dict={Z: fake_batch, X: batch}) for part in D_accur]

            print("Iter %05d: Losses: %.2f and %.2f"%(ITER, D_train_loss, G_train_loss))
            print("Accur: %.2f%% | Real: %.2f%% | Fake: %.2f%%" % (100 * D_train_accur[0], 100 * D_train_accur[1], 100 * D_train_accur[2]))
        
        summary_writer.add_summary(G_sample_image.eval(feed_dict={Z: fake_batch, X: batch}), ITER)
        summary_writer.flush()